# Implement the U-Net Model Architecture

In [1]:
from tensorflow import keras
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Input, ReLU, Cropping2D, Concatenate

In [17]:
width = 572

# the weight are initialized from a normal Gaussian distribution with std div sqrt(2/N) 
# where N the number of weights on that Convolution layer
# 
weight_initial = keras.initializers.VarianceScaling(scale=2, )

input = Input(shape=(width, width, 1))

#first layer of downgration
conv_left_a1 = Conv2D(filters=64, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(input)
conv_left_a2 = Conv2D(filters=64, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conv_left_a1)
# downgrade the image dimensions from 568, 588 to 392, 392 
# -> 176 less per dim -> crop 176/2 = 88 left, right, top, bottom 
conv_a_crop = Cropping2D(cropping=((88, 88), (88, 88)) )(conv_left_a2)
maxpool_a = MaxPooling2D(strides=2)(conv_left_a2)

#second layer of downgration
conv_left_b1 = Conv2D(filters=128, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(maxpool_a)
conv_left_b2 = Conv2D(filters=128, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conv_left_b1)
# downgrade the image dimensions from 280, 282 to 200, 200 
# -> 80 less per dim -> crop 40 left, right, top, bottom 
conv_b_crop = Cropping2D(cropping=((40, 40), (40, 40)) )(conv_left_b2)
maxpool_b = MaxPooling2D(strides=2)(conv_left_b2)

#third layer of downgration
conv_c1 = Conv2D(filters=256, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(maxpool_b)
conv_c2 = Conv2D(filters=256, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conv_c1)
# downgrade the image dimensions from 136, 136 to 104, 104 
# -> 32 less per dim -> crop 16 left, right, top, bottom 
conv_c_crop = Cropping2D(cropping=((16, 16), (16, 16)) )(conv_c2)
maxpool_c = MaxPooling2D(strides=2)(conv_c2)

# fourth layer of downgration
conv_left_d1 = Conv2D(filters=512, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(maxpool_c)
conv_left_d2 = Conv2D(filters=512, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conv_left_d1)
# downgrade the image dimensions from 64, 64 to 56, 56 
# -> 8 less per dim -> crop 4 left, right, top, bottom 
conv_d_crop = Cropping2D(cropping=((4, 4), (4, 4)) )(conv_left_d2)
maxpool_d = MaxPooling2D(strides=2)(conv_left_d2)

# fifth and bottom level
conv_e1 = Conv2D(filters=1024, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(maxpool_d)
conv_e2 = Conv2D(filters=1024, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conv_e1)
# input to layer fourth have 1024 filters after concatenation so use half for transpose convolution
up_conv_e = Conv2DTranspose(filters=512, kernel_size=2, strides=(2, 2))(conv_e2)

# upward to the fourth level
conct_d = Concatenate()([up_conv_e, conv_d_crop])
conv_right_d1 = Conv2D(filters=512, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conct_d)
conv_right_d2 = Conv2D(filters=512, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conv_right_d1)
up_conv_d = Conv2DTranspose(filters=256, kernel_size=2, strides=(2, 2))(conv_right_d2)

# m = keras.Model(inputs = input, outputs = up_conv_d)
# m.summary()

# # upward to the third level
conct_c = Concatenate()([up_conv_d, conv_c_crop])
conv_right_c1 = Conv2D(filters=256, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conct_c)
conv_right_c2 = Conv2D(filters=256, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conv_right_c1)
up_conv_c = Conv2DTranspose(filters=128, kernel_size=2, strides=(2, 2))(conv_right_c2)

# m = keras.Model(inputs = input, outputs = up_conv_c)
# m.summary()
# upward to the second level
conct_b = Concatenate()([up_conv_c, conv_b_crop])
conv_right_b1 = Conv2D(filters=128, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conct_b)
conv_right_b2 = Conv2D(filters=128, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conv_right_b1)
up_conv_b = Conv2DTranspose(filters=64, kernel_size=2, strides=(2, 2))(conv_right_b2)

# upward to the first level
conct_a = Concatenate()([up_conv_b, conv_a_crop])
conv_right_a1 = Conv2D(filters=64, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conct_a)
conv_right_a2 = Conv2D(filters=64, kernel_size=3, activation='relu', kernel_initializer=weight_initial)(conv_right_a1)
output = Conv2D(filters=2, kernel_size=1, activation='sigmoid', kernel_initializer=weight_initial)(conv_right_a2)

U_Net = keras.Model(inputs = input, outputs = output)
U_Net.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 572, 572,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_178 (Conv2D) │ (None, 570, 570,  │        640 │ input_layer_13[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_179 (Conv2D) │ (None, 568, 568,  │     36,928 │ conv2d_178[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_48    │ (None, 284, 284,  │          0 │ conv2d_179[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_180 (Conv2D) │ (None, 282, 282,  │     73,856 │ max_pooling2d_48… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_181 (Conv2D) │ (None, 280, 280,  │    147,584 │ conv2d_180[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_49    │ (None, 140, 140,  │          0 │ conv2d_181[0][0]  │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_182 (Conv2D) │ (None, 138, 138,  │    295,168 │ max_pooling2d_49… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_183 (Conv2D) │ (None, 136, 136,  │    590,080 │ conv2d_182[0][0]  │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_50    │ (None, 68, 68,    │          0 │ conv2d_183[0][0]  │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_184 (Conv2D) │ (None, 66, 66,    │  1,180,160 │ max_pooling2d_50… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_185 (Conv2D) │ (None, 64, 64,    │  2,359,808 │ conv2d_184[0][0]  │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_51    │ (None, 32, 32,    │          0 │ conv2d_185[0][0]  │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_186 (Conv2D) │ (None, 30, 30,    │  4,719,616 │ max_pooling2d_51… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_187 (Conv2D) │ (None, 28, 28,    │  9,438,208 │ conv2d_186[0][0]  │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_36 │ (None, 56, 56,    │  2,097,664 │ conv2d_187[0][0]  │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cropping2d_51       │ (None, 56, 56,    │          0 │ conv2d_185[0][0]

 Total params: 31,030,658 (118.37 MB)

 Trainable params: 31,030,658 (118.37 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
keras.utils.plot_model(U_Net, "my_first_model.png")

AttributeError: module 'pydot' has no attribute 'InvocationException'